# Lab 2


Written by Jimmy Phan, Yash Parakh, and Matthew Prinz

## 1. Correlations



*   read data in pandas AND seaborn



In [2]:
# from google.colab import files
# uploaded = files.upload()

import numpy as np
import pandas as pd
import io
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
# uploaded['DF1']
df1 = pd.read_csv(io.BytesIO(uploaded['DF1']), delimiter=',', names=['index', '0', '1', '2', '3']).drop(0).drop('index', axis=1)
df1.head()

NameError: name 'uploaded' is not defined

In [ ]:
# Seaborn implementation 
sns.pairplot(df1)
sns.despine()

In [ ]:
# Pandas implementation
for i in range(0, len(df1.columns)):
  for j in range(i + 1, len(df1.columns)):
    df1.plot.scatter(x=str(i), y=str(j), c='DarkBlue')



*   Compute covariance matrix



1.   Strong positive correlation between columns 2 and 0 (strong positive covariance of .99)
2.   Strong negative correlation between columns 1 and 3 (strong negative covariance of -0.99)





In [ ]:
# need to write explicit expression for this too
df1.cov()



*   Gaussian Random Vector



In [ ]:
mean = [0, 0, 0]
cov = [[1, 0, 0],
       [0, 1, 1],
       [0, 1, 1]]
samples = []
cov_list = []
num_samples = 1000
for i in range(num_samples):
  samples.append(list(np.random.multivariate_normal(mean, cov)))
  if i > 1:
    cov_list.append(pd.DataFrame(samples).cov().iloc[1][2])

sns.scatterplot(x=[i for i in range(len(cov_list))], y=cov_list)

In [ ]:
samples = []
for i in range(100):
  samples.append(list(np.random.multivariate_normal(mean, cov)))

pd.DataFrame(samples).cov()

## 2. Outliers

In [4]:
from scipy import stats
uploaded2 = files.upload()

NameError: name 'files' is not defined

In [ ]:
from numpy import linalg as la
df2 = pd.read_csv(io.BytesIO(uploaded2['DF2']), names=['index', '0', '1']).drop(0).drop('index', axis=1)
# df2.plot.scatter(x='0', y='1')
# slope, intercept, r_value, p_value, std_err = stats.linregress(df2.values[:,0],df2.values[:,1])
# line = slope*df2.values[:,0]+intercept
# plt.plot(df2.values[:,0],df2.values[:,1],'o',df2.values[:,0],line)

w, v = la.eig(df2.cov())
A = np.matmul(np.diag(np.power(w, -0.5)), v)
data_transformed = np.transpose(np.matmul(A, np.transpose(np.array(df2))))
plt.scatter(data_transformed[:,0], data_transformed[:,1], c='r')

The point at (-1, 1) in the original data is more "outling" as is seen in the normalized data above.

## 3. Even More Standard Error


In [ ]:
from sklearn.linear_model import LinearRegression

mean, variance = 0, 1
num_of_samples = [1, 5, 10, 30, 50, 100, 300, 500]
beta = [-3, 0]
b1_std, b2_std = [], []

# collect error samples for each n
for n in num_of_samples:
    error = np.empty((0, 2))
    for i in range(1000):   
        x_samples = np.random.normal(mean, variance, n).reshape(-1, 1)
        noise = np.random.normal(mean, variance, n).reshape(-1, 1)
        # generate samples of y
        y = (beta[0] + x_samples*beta[1] + noise).reshape(-1, 1)

        # create model to regress
        model = LinearRegression(fit_intercept=True)
        model.fit(x_samples, y)

        # record beta_hat values and error
        beta_hat = [model.intercept_[0], model.coef_[0][0]]
        error = np.append(error, (np.array(beta_hat) - np.array(beta)).reshape(1, 2), axis=0)
    
    # find the std of error
    beta0_std = np.var(error[:, 0])
    beta1_std = np.var(error[:, 1])
    b1_std.append(beta0_std)
    b2_std.append(beta1_std)

# plot each standard deviation
sns.lineplot(x=num_of_samples, y=b1_std)
x_ = np.arange(5,500)
sns.lineplot(x=x_, y=1/(np.sqrt(x_)))


## 4. Names and Frequencies


In [ ]:
# First bullet point
def return_top_k_names(year, k):
    file_name = "yob" + str(year) + ".txt"
    df = pd.read_csv(file_name, sep=",", header=None)
    for i in range(k):
        print(f"The {i+1} most popular name is: {df.iloc[i][0]}")
return_top_k_names(1880, 5)

In [ ]:
# Second bullet point
# This function is kind of dirty, but I don't know enough pandas to make it better.
def freq_men_women(name):
    frequency_m = 0
    frequency_f = 0
    for i in range(1888, 2016):
        file_name = "yob" + str(i) + ".txt"
        df = pd.read_csv(file_name, sep=",", header=None)
        for i, person in enumerate(df.values[:,0]):
            if name==person:
                if df.iloc[i, 1] == "F":
                    frequency_f += df.iloc[i, 2]
                elif df.iloc[i, 1] == "M":
                    frequency_m += df.iloc[i, 2]
                else:
                    print(f'{df.iloc[i, 1]}')
    print(f'{name} occurred in {frequency_m} males and in {frequency_f} females.')
    
freq_men_women("Mary")

In [ ]:
# Third bullet point
def relative_freq_men_women(name):
    frequency_m = 0
    frequency_f = 0
    total_m = 0
    total_f = 0
    for i in range(1888, 2016):
        file_name = "yob" + str(i) + ".txt"
        df = pd.read_csv(file_name, sep=",", header=None)
        for i, person in enumerate(df.values[:,0]):
            if df.iloc[i, 1] == "F":
                total_f += df.iloc[i, 2]
            if df.iloc[i, 1] == "M":
                total_m += df.iloc[i, 2]
            if name==person:
                if df.iloc[i, 1] == "F":
                    frequency_f += df.iloc[i, 2]
                elif df.iloc[i, 1] == "M":
                    frequency_m += df.iloc[i, 2]
                else:
                    print(f'ERROR WT')
    print(f'{name} occurred in {frequency_m/total_m} males and in {frequency_f/total_f} females.')
relative_freq_men_women("Matt")

In [ ]:
# Fourth bullet point
class Person:
    def __init__(self, name, gender):
        self.gender = gender
        self.name = name
    def __repr__(self):
        return self.name
    def __eq__(self, other):
        return self.name == other
    def __hash__(self):
        return hash(self.name) # This isn't ideal.
    def __ne__(self, other):
        return not(self == other)

def genderflops(year):
    """
    Finds all names that used to be more popular for one gender since year, but then became more popular for another gender since 
    given parameter year
    :param year: integer year, pre: 1888 <= year <= 2015
    """
    # Defining dictionaries
    names_and_frequencies_past_men = {}
    names_and_frequencies_future_men = {}
    names_and_frequencies_past_women = {}
    names_and_frequencies_future_women = {}
    
    # Iterating through all files up to user selected year
    for i in range(1888, year+1):
        file_name = "yob" + str(i) + ".txt"
        df = pd.read_csv(file_name, sep=",", header=None)
        for i in range(len(df.values[:, 0])):
            sex = df.iloc[i, 1]
            name = df.iloc[i, 0]
            frequency = df.iloc[i, 2]
            # Add men to mens dictionary and women to women's dictionary
            if sex == "M":
                if name in names_and_frequencies_past_men.keys():
                    names_and_frequencies_past_men[Person(name, sex)] += frequency
                else:
                    names_and_frequencies_past_men[Person(name, sex)] = frequency
            elif sex == "F":
                if name in names_and_frequencies_past_women.keys():
                    names_and_frequencies_past_women[Person(name, sex)] += frequency
                else:
                    names_and_frequencies_past_women[Person(name, sex)] = frequency
    # Iterating through all files from user selected year to 2015
    for i in range(year, 2016):
        file_name = "yob" + str(i) + ".txt"
        df = pd.read_csv(file_name, sep=",", header=None)
        for i in range(len(df.values[:, 0])):
            sex = df.iloc[i, 1]
            name = df.iloc[i, 0]
            frequency = df.iloc[i, 2]
            # Add men to mens dictionary and women to women's dictionary
            if sex == "M":
                if name in names_and_frequencies_future_men.keys():
                    names_and_frequencies_future_men[Person(name, sex)] += frequency
                else:
                    names_and_frequencies_future_men[Person(name, sex)] = frequency
            elif sex == "F":
                if name in names_and_frequencies_future_women.keys():
                    names_and_frequencies_future_women[Person(name, sex)] += frequency
                else:
                    names_and_frequencies_future_women[Person(name, sex)] = frequency
                    
    # Genderflop 1: Used to be more popular for men, then it became more popular for women
    for past_name, past_male_frequency in names_and_frequencies_past_men.items():
        if past_name in names_and_frequencies_past_women.keys():
            try:
                past_female_frequency = names_and_frequencies_past_women[past_name]
                current_female_frequency = names_and_frequencies_future_women[past_name]
                current_male_frequency = names_and_frequencies_future_men[past_name]
            except KeyError:
                continue
            if past_male_frequency > past_female_frequency and current_female_frequency > current_male_frequency:
                print(f'From 1888-{year}, {past_name} was more popular for men ({past_male_frequency} > {past_female_frequency}), but from {year}-2015 is more popular for women. ({current_female_frequency} > {current_male_frequency})')
    
    # Genderflop 2: Used to be more popular for women, then it became more popular for men
    for past_name, past_male_frequency in names_and_frequencies_past_men.items():
        if past_name in names_and_frequencies_past_women.keys():
            try:
                past_female_frequency = names_and_frequencies_past_women[past_name]
                current_female_frequency = names_and_frequencies_future_women[past_name]
                current_male_frequency = names_and_frequencies_future_men[past_name]
            except KeyError:
                continue
            if past_female_frequency > past_male_frequency and current_male_frequency > current_female_frequency:
                print(f'From 1888-{year}, {past_name} was more popular for women ({past_female_frequency} > {past_male_frequency}), but from {year}-2015 is more popular for men. ({current_male_frequency} > {current_female_frequency})')

genderflops(1960) 

In [ ]:
# Fifth bullet point. 
# Cool stuff about the data set - seeing what became more popular 
class Person:
    def __init__(self, name, gender):
        self.gender = gender
        self.name = name
    def __repr__(self):
        return self.name
    def __eq__(self, other):
        return self.name == other
    def __hash__(self):
        return hash(self.name + self.gender)
    def __ne__(self, other):
        return not(self == other)
def past_vs_year_frequency_differences(year):
    """
    Finds names that increased or decreased in frequency since given parameter year.
    :param year: integer year, pre: 1888 <= year <= 2015
    """
    names_and_frequencies_past = {}
    names_and_frequencies_future = {}
    for i in range(1888, year+1):
        file_name = "yob" + str(i) + ".txt"
        df = pd.read_csv(file_name, sep=",", header=None)
        for i in range(len(df.values[:, 0])):
            sex = df.iloc[i, 1]
            name = df.iloc[i, 0]
            frequency = df.iloc[i, 2]
            if name in names_and_frequencies_past.keys():
                names_and_frequencies_past[Person(name, sex)] += frequency
            else:
                names_and_frequencies_past[Person(name, sex)] = frequency
                
    for i in range(year, 2016):
        file_name = "yob" + str(i) + ".txt"
        df = pd.read_csv(file_name, sep=",", header=None)
        for i in range(len(df.values[:, 0])):
            sex = df.iloc[i, 1]
            name = df.iloc[i, 0]
            frequency = df.iloc[i, 2]
            if name in names_and_frequencies_future.keys():
                names_and_frequencies_future[Person(name, sex)] += frequency
            else:
                names_and_frequencies_future[Person(name, sex)] = frequency
                
    for k, v in names_and_frequencies_past.items():
        if k in names_and_frequencies_future.keys():
            current_val = names_and_frequencies_future[k]
            if v > current_val:
                print(f"{k} occured more in the past, with past rate: {v} and current_rate {current_val}")
            elif v == current_val:
                print(f"{k} occurs at equal frequency in past and future, with past rate: {v} and current_rate {current_val}")
            elif v < current_val:
                print(f"{k} occured more in the future, with past rate: {v} and current_rate {current_val}")
past_vs_year_frequency_differences(1952) # approximately halfway, time-wise. doesn't account for more people being born.

## Regression and Interaction Terms

In [ ]:
from sklearn import linear_model

In [ ]:
kidiq = pd.read_stata("kidiq.dta")
interaction_term = kidiq.as_matrix(["mom_hs"]) * kidiq.as_matrix(["mom_iq"])
kidiq["mom_iq*mom_hs"] = interaction_term
kidiq.head(10)

In [ ]:
sns.pairplot(kidiq)

In [ ]:
kidiq_x_with_it = kidiq.as_matrix(["mom_iq","mom_hs","mom_iq*mom_hs"])
kidiq_y_with_it = kidiq.as_matrix(["kid_score"])
lr = linear_model.LinearRegression()
lr.fit(kidiq_x_with_it, kidiq_y_with_it)
kidiq_x = kidiq.as_matrix(["mom_iq","mom_hs"])
kidiq_y = kidiq.as_matrix(["kid_score"])
lr2 = linear_model.LinearRegression()
lr2.fit(kidiq_x, kidiq_y)

In [ ]:
print(f'With the interactive term: This is the slope: {lr.coef_}. This is the intercept: {lr.intercept_}. This is the R^2: {lr.score(kidiq_x_with_it, kidiq_y_with_it)}.')
print(f'Without the interactive term: This is the slope: {lr2.coef_}. This is the intercept: {lr2.intercept_}. This is the R^2: {lr2.score(kidiq_x, kidiq_y)}.')

The $R^2$ term with the interaction term increases from 0.214 to 0.230. That is a sign that the interaction term should be there. That means that there is a relationship between 

In [ ]:
x = kidiq.values[:,2]
y = kidiq.values[:,0]
m1 = lr.coef_[0][0]
b1 = lr.intercept_
m2 = lr2.coef_[0][0]
b2 = lr2.intercept_
plt.scatter(x, y)
plt.plot(x, m1*x+b1, '-',c='g')
plt.plot(x, m2*x+b2, '-', c='r')
plt.xlabel("Mom IQ")
plt.ylabel("Kid Score")
plt.title("Mom IQ vs. Kid Score")

We can see a visualization of the linear regressions using the green and red line. The green line comes from the model with the interaction term, whereas the red line comes from model without the interaction term. 

The green line seems to be a better fit, further suggesting that the interaction term should be there.